In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-cma_u5d5
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-cma_u5d5
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=d8d199a7fefbe75e9acb54a5982460d2d60ec2048b66359c9fc14d6dd3671c1c
  Stored in directory: /tmp/pip-ephem-wheel-cache-60pka5qf/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [5]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <limits.h>
#include <time.h>

#define V 5
#define BLOCK_SIZE 32

__global__ void dijkstra_kernel(int *graph, int *dist, int *sptSet, int *min_index)
{
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < V) {
        if (sptSet[tid] == 0 && dist[tid] < dist[*min_index])
            *min_index = tid;
    }

    __syncthreads();

    int u = *min_index;

    sptSet[u] = 1;

    for (int v = 0; v < V; v++)
        if (!sptSet[v] && graph[u * V + v] && dist[u] != INT_MAX
            && dist[u] + graph[u * V + v] < dist[v])
            dist[v] = dist[u] + graph[u * V + v];
}

void dijkstra(int *graph, int src)
{
    int *dist, *sptSet, *min_index;
    int *d_graph, *d_dist, *d_sptSet, *d_min_index;

    dist = (int*)malloc(V * sizeof(int));
    sptSet = (int*)malloc(V * sizeof(int));
    min_index = (int*)malloc(sizeof(int));

    for (int i = 0; i < V; i++)
        dist[i] = INT_MAX, sptSet[i] = 0;

    dist[src] = 0;

    cudaMalloc(&d_graph, V * V * sizeof(int));
    cudaMalloc(&d_dist, V * sizeof(int));
    cudaMalloc(&d_sptSet, V * sizeof(int));
    cudaMalloc(&d_min_index, sizeof(int));

    cudaMemcpy(d_graph, graph, V * V * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_dist, dist, V * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_sptSet, sptSet, V * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_min_index, min_index, sizeof(int), cudaMemcpyHostToDevice);

    clock_t start_time = clock();

    for (int count = 0; count < V - 1; count++) {
        *min_index = INT_MAX;

        dijkstra_kernel<<<(V + BLOCK_SIZE - 1) / BLOCK_SIZE, BLOCK_SIZE>>>(d_graph, d_dist, d_sptSet, d_min_index);

        cudaMemcpy(min_index, d_min_index, sizeof(int), cudaMemcpyDeviceToHost);
    }

    clock_t end_time = clock();

    double time_taken = ((double)(end_time - start_time)) / CLOCKS_PER_SEC;

    cudaMemcpy(dist, d_dist, V * sizeof(int), cudaMemcpyDeviceToHost);

    printf("Vertex \t Distance from Source\n");
    for (int i = 0; i < V; i++)
        printf("%d \t %d\n", i, dist[i]);

    printf("Time taken: %lf seconds\n", time_taken);

    free(dist);
    free(sptSet);
    free(min_index);
    cudaFree(d_graph);
    cudaFree(d_dist);
    cudaFree(d_sptSet);
    cudaFree(d_min_index);
}
int main()
{
int graph[V][V] = {{0, 10, 20, 0, 0},
{10, 0, 5, 15, 0},
{20, 5, 0, 20, 30},
{0, 15, 20, 0, 35},
{0, 0, 30, 35, 0}};
 dijkstra((int*)graph, 0);

return 0;
 }

Vertex 	 Distance from Source
0 	 0
1 	 10
2 	 20
3 	 2147483647
4 	 2147483647
Time taken: 0.000105 seconds



In [8]:
%%cu
int main() {
    cudaFileHandle_t input_file, output_file;
    char ch;
    clock_t start_time = clock();

    // Open the input file in read mode
    if (cuFileOpen(&input_file, "Cars.txt", CU_FILE_READ) != CUDA_SUCCESS) {
        printf("Error opening input file.\n");
        return 1;
    }

    // Open the output file in write mode
    if (cuFileOpen(&output_file, "output.txt", CU_FILE_WRITE) != CUDA_SUCCESS) {
        printf("Error opening output file.\n");
        return 1;
    }

    // Measure the start time
   

    // Read from the input file character by character and write to the output file
    while ((cuFileRead(&ch, sizeof(char), 1, input_file)) == 1) {
        cuFileWrite(&ch, sizeof(char), 1, output_file);
    }

    // Measure the end time
   

    // Close the files
    cuFileClose(input_file);
    cuFileClose(output_file);
    clock_t end_time = clock();

    // Calculate the time taken
    double time_taken = (double)(end_time - start_time) / CLOCKS_PER_SEC;

    printf("File copied successfully in %lf seconds.\n", time_taken);

    return 0;
}

/tmp/tmpcxhi9n6i/f7449a62-dd32-4df9-8878-4bcbec4483fd.cu(2): error: identifier "cudaFileHandle_t" is undefined

/tmp/tmpcxhi9n6i/f7449a62-dd32-4df9-8878-4bcbec4483fd.cu(7): error: identifier "CU_FILE_READ" is undefined

/tmp/tmpcxhi9n6i/f7449a62-dd32-4df9-8878-4bcbec4483fd.cu(7): error: identifier "cuFileOpen" is undefined

/tmp/tmpcxhi9n6i/f7449a62-dd32-4df9-8878-4bcbec4483fd.cu(7): error: identifier "CUDA_SUCCESS" is undefined

/tmp/tmpcxhi9n6i/f7449a62-dd32-4df9-8878-4bcbec4483fd.cu(13): error: identifier "CU_FILE_WRITE" is undefined

/tmp/tmpcxhi9n6i/f7449a62-dd32-4df9-8878-4bcbec4483fd.cu(22): error: identifier "cuFileRead" is undefined

/tmp/tmpcxhi9n6i/f7449a62-dd32-4df9-8878-4bcbec4483fd.cu(23): error: identifier "cuFileWrite" is undefined

/tmp/tmpcxhi9n6i/f7449a62-dd32-4df9-8878-4bcbec4483fd.cu(30): error: identifier "cuFileClose" is undefined

8 errors detected in the compilation of "/tmp/tmpcxhi9n6i/f7449a62-dd32-4df9-8878-4bcbec4483fd.cu".

